<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [2]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [26]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [25]:
# use requests.get() method with the provided static_url
# assign the response to a object
response=requests.get(static_url)

<Response [200]>

Create a `BeautifulSoup` object from the HTML `response`


In [27]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.content, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [ ]:
# Use soup.title attribute
print(soup.prettify())

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [12]:
th_elements = first_launch_table.find_all('th')
column_names = []
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [13]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [50]:
launch_dict= dict.fromkeys(column_names)
# Remove an irrelvant column
del launch_dict['Date and time ( )']
# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [51]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            datatimelist=date_time(row[0])
            launch_dict['Flight No.'] = launch_dict.get('Flight No.', []) + [flight_number]
            #print(flight_number)
            print(flight_number)
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = launch_dict.get('Date', []) + [date]
            #print(date)
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
            launch_dict['Time'] = launch_dict.get('Time', []) + [time]
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            launch_dict['Version Booster'] = launch_dict.get('Version Booster', []) + [bv]
            if not(bv):
                bv=row[1].a.string
            print(bv)
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'] = launch_dict.get('Launch site', []) + [launch_site]
            #print(launch_site)
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'] = launch_dict.get('Payload', []) + [payload] 
            #print(payload)
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'] = launch_dict.get('Payload mass', []) + [payload_mass]

            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'] = launch_dict.get('Orbit', []) + [orbit]
            #print(orbit)
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            try:
                customer = row[6].a.string if row[6].a else row[6].get_text(strip=True)
            except IndexError:
                customer = None
            launch_dict['Customer'] = launch_dict.get('Customer', []) + [customer]
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = launch_dict.get('Launch outcome', []) + [launch_outcome]
            #print(launch_outcome)

            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'] = launch_dict.get('Booster landing', []) + [booster_landing]

            #print(booster_landing)
            

1
F9 v1.0B0003.1
2
F9 v1.0B0004.1
3
F9 v1.0B0005.1
4
F9 v1.0B0006.1
5
F9 v1.0B0007.1
6
F9 v1.1B1003
7
F9 v1.1
8
F9 v1.1
9
F9 v1.1
10
F9 v1.1
11
F9 v1.1
12
F9 v1.1
13
F9 v1.1
14
F9 v1.1
15
F9 v1.1
16
F9 v1.1
17
F9 v1.1
18
F9 v1.1
19
F9 v1.1
20
F9 FT
21
F9 v1.1
22
F9 FT
23
F9 FT
24
F9 FT
25
F9 FT
26
F9 FT
27
F9 FT
28
F9 FT
29
F9 FT
30
F9 FT
31
F9 FT
32
F9 FT♺
33
F9 FT
34
F9 FT
35
F9 FT
36
F9 FTB1029.2
37
F9 FT
38
F9 FT
39
F9 B4
40
F9 FT
41
F9 B4
42
F9 B4
43
F9 FTB1031.2
44
F9 B4
45
F9 FTB1035.2
46
F9 FTB1036.2
47
F9 B4
48
F9 FTB1032.2
49
F9 FTB1038.2
50
F9 B4
51
F9 B4B1041.2
52
F9 B4B1039.2
53
F9 B4
54
F9 B5B1046.1
55
F9 B4B1043.2
56
F9 B4B1040.2
57
F9 B4B1045.2
58
F9 B5
59
F9 B5B1048
60
F9 B5B1046.2
61
F9 B5
62
F9 B5B1048.2
63
F9 B5B1047.2
64
F9 B5B1046.3
65
F9 B5
66
F9 B5
67
F9 B5B1049.2
68
F9 B5B1048.3
69
F9 B5[268]
70
F9 B5
71
F9 B5B1049.3
72
F9 B5B1051.2
73
F9 B5B1056.2
74
F9 B5B1047.3
75
F9 B5
76
F9 B5
77
F9 B5B1056.3
78
F9 B5
79
F9 B5
80
F9 B5
81
F9 B5
82
F9 B5
83
F9 B5
84
F9 B5
8

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [49]:
df = pd.DataFrame({key:pd.Series(value, dtype='object') for key, value in launch_dict.items()})
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,Launch Site,PayLoad,PadLoad mass,Booster Landing
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45,CCAFS,Dragon Spacecraft Qualification Unit,"7,000 kg",Failure
1,1,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0003.1,Failure,4 June 2010,18:45,CCAFS,Dragon,NaN,Failure
2,2,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0004.1,No attempt\n,8 December 2010,15:43,CCAFS,Dragon,NaN,No attempt\n
3,3,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0005.1,No attempt,22 May 2012,07:44,CCAFS,SpaceX CRS-1,NaN,No attempt
4,4,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt\n,8 October 2012,00:35,CCAFS,SpaceX CRS-2,NaN,No attempt\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,117,NaN,NaN,NaN,LEO,Sirius XM,Success\n,F9 B5B1051.10,NaN,9 May 2021,06:42,NaN,NaN,NaN,NaN
799,118,NaN,NaN,NaN,LEO,NaN,NaN,F9 B5B1058.8,NaN,15 May 2021,22:56,NaN,NaN,NaN,NaN
800,119,NaN,NaN,NaN,LEO,NaN,NaN,F9 B5B1063.2,NaN,26 May 2021,18:59,NaN,NaN,NaN,NaN
801,120,NaN,NaN,NaN,GTO,NaN,NaN,F9 B5B1067.1,NaN,3 June 2021,17:29,NaN,NaN,NaN,NaN


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
